In [1]:
from mangaki.algo.dataset import Dataset
from mangaki.utils.values import rating_values

queryset = Rating.objects.all()  #.exclude(choice__in=['willsee', 'wontsee']) # Security if the dataset is public
triplets = queryset.values_list('user_id', 'work_id', 'choice')
titles = Work.objects.order_by('id').values_list('id', 'title')
categories = Work.objects.order_by('id').values_list('id', 'category__slug')
dataset = Dataset()

In [2]:
anonymized = dataset.make_anonymous_data(triplets, convert=lambda choice: rating_values[choice], ordered=True)

In [3]:
dataset.anonymized.y

array([-0.5,  0.1,  0.1, ..., -2. ,  2. ,  2. ])

In [4]:
dataset.titles = dict(titles)
dataset.categories = dict(categories)

In [5]:
dataset.save_csv(suffix='-ordered')

In [6]:
dataset.save('tsne.pickle')

In [9]:
import pandas as pd

In [11]:
import numpy as np

arr = np.random.random((4, 2))

In [17]:
pd.DataFrame(arr).to_csv

<bound method DataFrame.to_csv of           0         1
0  0.236613  0.374644
1  0.232992  0.542192
2  0.847818  0.977698
3  0.723179  0.016853>

In [16]:
import os

os.system('cat test.csv')

0

In [19]:
pd.DataFrame(arr).to_csv('test.csv', header=False, index=False)

In [24]:
np.column_stack((arr, np.ones(4)))

array([[ 0.23661292,  0.37464382,  1.        ],
       [ 0.23299154,  0.54219249,  1.        ],
       [ 0.84781838,  0.97769781,  1.        ],
       [ 0.7231793 ,  0.01685298,  1.        ]])

In [28]:
np.hstack((arr, np.ones(4).reshape(4, 1)))

array([[ 0.23661292,  0.37464382,  1.        ],
       [ 0.23299154,  0.54219249,  1.        ],
       [ 0.84781838,  0.97769781,  1.        ],
       [ 0.7231793 ,  0.01685298,  1.        ]])